In [118]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

In [119]:
hotels = pd.read_csv('data/hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [122]:
# ищем признаки с пропусками
hotels.isna().sum()

hotel_address                                    0
additional_number_of_scoring                     0
review_date                                      0
average_score                                    0
hotel_name                                       0
reviewer_nationality                             0
negative_review                                  0
review_total_negative_word_counts                0
total_number_of_reviews                          0
positive_review                                  0
review_total_positive_word_counts                0
total_number_of_reviews_reviewer_has_given       0
reviewer_score                                   0
tags                                             0
days_since_review                                0
lat                                           2448
lng                                           2448
dtype: int64

In [49]:
# ищем имена отелей, у которых не заполнены координаты
hotel_without_lat_lng = hotels[hotels['lat'].isna()][['hotel_name']].value_counts().to_frame()
hotel_without_lat_lng

,0
hotel_name,
Fleming s Selection Hotel Wien City,496
Hotel City Central,421
Hotel Atlanta,285
Maison Albar Hotel Paris Op ra Diamond,224
Hotel Daniel Vienna,182
Hotel Pension Baron am Schottentor,172
Austria Trend Hotel Schloss Wilhelminenberg Wien,141
NH Collection Barcelona Podium,113
Derag Livinghotel Kaiser Franz Joseph Vienna,112


In [124]:
# проверяем, что у выбранных отелей везде не заполнены координаты
# если бы это было не так, то мы могли бы взять координаты из существующих
check_null_hotels = hotels.merge(hotel_without_lat_lng, left_on='hotel_name', right_index=True, how='inner')
check_null_hotels[['lat','lng']].isna().value_counts()
# check_null_hotels[['hotel_name','hotel_address']].value_counts()

lat   lng 
True  True    2448
dtype: int64

In [126]:
hotels_coordinates = {
    'Fleming s Selection Hotel Wien City':[48.209367, 16.353385],
    'Hotel City Central':[48.213591, 16.379863],
    'Hotel Atlanta':[48.220412, 16.355809],
    'Maison Albar Hotel Paris Op ra Diamond':[48.875273, 2.323392],
    'Hotel Daniel Vienna':[48.188799, 16.383790],
    'Hotel Pension Baron am Schottentor':[48.216802, 16.359905],
    'Austria Trend Hotel Schloss Wilhelminenberg Wien':[48.219567, 16.285596],
    'NH Collection Barcelona Podium':[41.391548, 2.177888],
    'Derag Livinghotel Kaiser Franz Joseph Vienna':[48.245983, 16.341410],
    'City Hotel Deutschmeister':[48.220882, 16.366644],
    'Holiday Inn Paris Montmartre':[48.888917, 2.333092],
    'Hotel Park Villa':[48.233514, 16.345660],
    'Cordial Theaterhotel Wien':[48.209546, 16.351453],
    'Roomz Vienna':[48.186810, 16.420713],
    'Mercure Paris Gare Montparnasse':[48.839995, 2.323561],
    'Hotel Advance':[41.385831, 2.154189],
    'Renaissance Barcelona Hotel':[41.392730, 2.167391]
}

hotels['lat'] = hotels.apply(lambda row: 
    hotels_coordinates.get(row['hotel_name'])[0] if row['hotel_name'] in hotels_coordinates else row['lat'],
    axis=1
    )

hotels['lng'] = hotels.apply(lambda row: 
    hotels_coordinates.get(row['hotel_name'])[1] if row['hotel_name'] in hotels_coordinates else row['lng'],
    axis=1
    )

hotels.isna().sum()

hotel_address                                 0
additional_number_of_scoring                  0
review_date                                   0
average_score                                 0
hotel_name                                    0
reviewer_nationality                          0
negative_review                               0
review_total_negative_word_counts             0
total_number_of_reviews                       0
positive_review                               0
review_total_positive_word_counts             0
total_number_of_reviews_reviewer_has_given    0
reviewer_score                                0
tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
dtype: int64

In [140]:
# визуально проверяем несколько значений    
hotels.loc[hotels['hotel_name'].isin(hotels_coordinates.keys())][['hotel_name', 'lat', 'lng']]

,hotel_name,lat,lng
122,Austria Trend Hotel Schloss Wilhelminenberg Wien,48.219567,16.285596
566,Holiday Inn Paris Montmartre,48.888917,2.333092
724,Fleming s Selection Hotel Wien City,48.209367,16.353385
754,Hotel Atlanta,48.220412,16.355809
1137,Maison Albar Hotel Paris Op ra Diamond,48.875273,2.323392
...,...,...,...
386092,Hotel City Central,48.213591,16.379863
386465,Hotel City Central,48.213591,16.379863
386504,Maison Albar Hotel Paris Op ra Diamond,48.875273,2.323392
386702,Hotel City Central,48.213591,16.379863


In [3]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)

In [4]:
# # Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# # Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
# X = hotels.drop(['reviewer_score'], axis = 1)  
# y = hotels['reviewer_score'] 

In [5]:
# # Загружаем специальный инструмент для разбивки:  
# from sklearn.model_selection import train_test_split  

In [6]:
# # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# # Для тестирования мы будем использовать 25% от исходного датасета.  
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [7]:
# # Импортируем необходимые библиотеки:  
# from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
# from sklearn import metrics # инструменты для оценки точности модели  
  
# # Создаём модель  
# regr = RandomForestRegressor(n_estimators=100)  
      
# # Обучаем модель на тестовом наборе данных  
# regr.fit(X_train, y_train)  
      
# # Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# # Предсказанные значения записываем в переменную y_pred  
# y_pred = regr.predict(X_test)  


In [8]:
# # Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# # Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
# print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.1412825587975143


Небольшой бонус:


In [9]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)